# RICERCA INFORMATA

### Librerie utilizzate

In [2]:
import osmnx as ox
import folium
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import heapq
import time
import sys
from geopy.distance import geodesic

### EURISTICHE UTILIZZATE PER A*

In [7]:
def geodetic_distance(graph, node1, node2):
    """Restituisce la distanza geodetica tra due nodi in metri."""
    lat1, lon1 = graph.nodes[node1]['y'], graph.nodes[node1]['x']
    lat2, lon2 = graph.nodes[node2]['y'], graph.nodes[node2]['x']
    return geodesic((lat1, lon1), (lat2, lon2)).meters

def incidenti_medi_per_km(graph):
    """Calcola il numero medio di incidenti per chilometro nel grafo."""
    total_incidents = sum(int(data.get("incidenti", 0)) for u, v, data in graph.edges(data=True))
    total_length = sum(float(data.get("length", 1)) for u, v, data in graph.edges(data=True)) / 1000  # Converti in km
    return total_incidents / total_length if total_length > 0 else 0

def lunghezza_media_archi(graph):
    """Calcola la lunghezza media degli archi nel grafo."""
    edge_lengths = [float(data.get("length", 1)) for u, v, data in graph.edges(data=True)]
    return sum(edge_lengths) / len(edge_lengths) if edge_lengths else 1

def min_incident_rate_per_km(graph):
    min_rate = float('inf')
    for u, v, data in graph.edges(data=True):
        length_km = float(data.get("length", 1)) / 1000  # Converti in km
        incidents = int(data.get("incidenti", 0))
        if length_km == 0:
            continue  # Evita divisione per zero
        rate = incidents / length_km
        if rate < min_rate:
            min_rate = rate
    return min_rate if min_rate != float('inf') else 0


def heuristic_1(graph, node, goal):
    """Calcola l'euristica h(n) per il nodo corrente basata su distanza e incidenti."""
    distanza = geodetic_distance(graph, node, goal)
    incidenti_km = incidenti_medi_per_km(graph)
    lunghezza_media = lunghezza_media_archi(graph)
    return distanza * (incidenti_km / lunghezza_media)

def heuristic_2(graph, node, goal, min_rate):
    distance_meters = geodetic_distance(graph, node, goal)
    distance_km = distance_meters / 1000
    return distance_km * min_rate

# A* con ***euristica ammissibile***

In [17]:
def a_star_search(graph, start, goal, min_rate):
    """Implementazione dell'algoritmo A* per minimizzare il numero di incidenti."""
    priority_queue = [(0, start, 0, [start])]  # (f(n), nodo, g(n), percorso)
    visited = {}
    explored_paths = []  # Ora è una lista di percorsi esplorati
    start_time = time.time()
    memory_before = sys.getsizeof(priority_queue) + sys.getsizeof(visited)
    max_depth = 0
    
    while priority_queue:
        f, node, g, path = heapq.heappop(priority_queue)  # Estrai il nodo con il miglior f(n)
        explored_paths.append(path)  # Salva il percorso esplorato
        max_depth = max(max_depth, len(path) - 1)
        
        if node == goal:
            execution_time = time.time() - start_time
            memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(visited)
            memory_used = memory_after - memory_before
            return path, g, explored_paths, execution_time, memory_used, max_depth  # Restituisce i percorsi esplorati
        
        if node in visited and g >= visited[node]:
            continue  # Ignora percorsi peggiori
        
        visited[node] = g  # Memorizza il miglior costo trovato per questo nodo
        
        for neighbor in graph.neighbors(node):
            edge_data = graph.get_edge_data(node, neighbor)
            
            if isinstance(edge_data, dict):
                edge_data = min(edge_data.values(), key=lambda d: int(d.get("incidenti", float('inf'))))
            
            edge_cost = int(edge_data.get("incidenti", 0))
            new_g = g + edge_cost
            h = heuristic_2(graph, neighbor, goal, min_rate)
            f = new_g + h
            
            heapq.heappush(priority_queue, (f, neighbor, new_g, path + [neighbor]))
    
    execution_time = time.time() - start_time
    memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(visited)
    memory_used = memory_after - memory_before
    return None, float('inf'), explored_paths, execution_time, memory_used, max_depth  # Ora explored_paths è una lista


# Caricare il grafo con il numero di incidenti sugli archi
G = ox.load_graphml("rete_bari_incidenti.graphml")

partenza = 270659688
arrivo = 1481415203

# Heuristic 2
min_rate = min_incident_rate_per_km(G)
shortest_path, min_incidents, explored_paths, exec_time, memory_used, max_depth = a_star_search(G, partenza, arrivo, min_rate)

if shortest_path:
    print(f"Percorso con il minor numero di incidenti trovato: {shortest_path}")
    print(f"\nNumero totale di incidenti lungo il percorso: {min_incidents}")
    print(f"\nNumero di percorsi visitati: {len(explored_paths)}")
    print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
    print(f"Memoria utilizzata: {memory_used} byte")
    print(f"Profondità massima raggiunta: {max_depth}")
else:
    print("Nessun percorso trovato.")
    print(f"Numero di percorsi visitati: {explored_paths}")
    print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
    print(f"Memoria utilizzata: {memory_used} byte")
    print(f"Profondità massima raggiunta: {max_depth}")

# RISULTATI OTTENUTI
# Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 12083787147, 569212252, 3860917204, 3841272949, 3860940020, 330076413, 279650482, 1483634369, 1483634284, 279655605, 280884748, 279380962, 339607009, 279655525, 10680697917, 316572227, 10680697906, 279655521, 9602691583, 1014703348, 1249973250, 4395827572, 5395447618, 5395447592, 5395448236, 5395447590, 330932109, 1459966804, 298502133, 329994613, 330003439, 1481415203]

# Numero totale di incidenti lungo il percorso: 123

# Numero di percorsi visitati: 6239
# Tempo di esecuzione: 1.5907 secondi
# Memoria utilizzata: 152208 byte
# Profondità massima raggiunta: 129

Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 12083787147, 569212252, 3860917204, 3841272949, 3860940020, 330076413, 279650482, 1483634369, 1483634284, 279655605, 280884748, 279380962, 339607009, 279655525, 10680697917, 316572227, 10680697906, 279655521, 9602691583, 1014703348, 1249973250, 4395827572, 5395447618, 5395447592, 5395448236, 5395447590, 330932109, 1459966804, 298502133, 329994613, 330003439, 1481415203]

Numero totale di incidenti lungo il percorso: 123

Numero di percorsi visitati: 6239
Tempo di esecuzione: 1.5907 secondi
Memoria utilizzata: 152208 byte
Profondità massima raggiunta: 129


# A* con ***euristica non ammissibile***

In [16]:

def a_star_search(graph, start, goal):
    """Implementazione dell'algoritmo A* per minimizzare il numero di incidenti."""
    priority_queue = [(0, start, 0, [start])]  # (f(n), nodo, g(n), percorso)
    visited = {}
    explored_paths = []  # Ora è una lista di percorsi esplorati
    start_time = time.time()
    memory_before = sys.getsizeof(priority_queue) + sys.getsizeof(visited)
    max_depth = 0
    
    while priority_queue:
        f, node, g, path = heapq.heappop(priority_queue)  # Estrai il nodo con il miglior f(n)
        explored_paths.append(path)  # Salva il percorso esplorato
        max_depth = max(max_depth, len(path) - 1)
        
        if node == goal:
            execution_time = time.time() - start_time
            memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(visited)
            memory_used = memory_after - memory_before
            return path, g, explored_paths, execution_time, memory_used, max_depth  # Restituisce i percorsi esplorati
        
        if node in visited and g >= visited[node]:
            continue  # Ignora percorsi peggiori
        
        visited[node] = g  # Memorizza il miglior costo trovato per questo nodo
        
        for neighbor in graph.neighbors(node):
            edge_data = graph.get_edge_data(node, neighbor)
            
            if isinstance(edge_data, dict):
                edge_data = min(edge_data.values(), key=lambda d: int(d.get("incidenti", float('inf'))))
            
            edge_cost = int(edge_data.get("incidenti", 0))
            new_g = g + edge_cost
            h = heuristic_1(graph, neighbor, goal)
            f = new_g + h
            
            heapq.heappush(priority_queue, (f, neighbor, new_g, path + [neighbor]))
    
    execution_time = time.time() - start_time
    memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(visited)
    memory_used = memory_after - memory_before
    return None, float('inf'), explored_paths, execution_time, memory_used, max_depth  # Ora explored_paths è una lista


# Caricare il grafo con il numero di incidenti sugli archi
G = ox.load_graphml("rete_bari_incidenti.graphml")

partenza = 270659688
arrivo = 1481415203

# Heuristic 1
shortest_path, min_incidents, explored_paths, exec_time, memory_used, max_depth = a_star_search(G, partenza, arrivo)


if shortest_path:
    print(f"Percorso con il minor numero di incidenti trovato: {shortest_path}")
    print(f"\nNumero totale di incidenti lungo il percorso: {min_incidents}")
    print(f"\nNumero di percorsi visitati: {len(explored_paths)}")
    print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
    print(f"Memoria utilizzata: {memory_used} byte")
    print(f"Profondità massima raggiunta: {max_depth}")
else:
    print("Nessun percorso trovato.")
    print(f"Numero di percorsi visitati: {explored_paths}")
    print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
    print(f"Memoria utilizzata: {memory_used} byte")
    print(f"Profondità massima raggiunta: {max_depth}")

# RISULTATI OTTENUTI
# Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 322548997, 270388924, 270388545, 330040330, 298504324, 298504314, 2882564567, 298504321, 10604170442, 11476336896, 270434053, 329994353, 270433936, 1108920034, 320970982, 320970994, 270654807, 270655162, 329988598, 329988600, 270655174, 270654641, 1481415203]

# Numero totale di incidenti lungo il percorso: 211

# Numero di percorsi visitati: 31
# Tempo di esecuzione: 1.6824 secondi
# Memoria utilizzata: 1352 byte
# Profondità massima raggiunta: 26

Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 322548997, 270388924, 270388545, 330040330, 298504324, 298504314, 2882564567, 298504321, 10604170442, 11476336896, 270434053, 329994353, 270433936, 1108920034, 320970982, 320970994, 270654807, 270655162, 329988598, 329988600, 270655174, 270654641, 1481415203]

Numero totale di incidenti lungo il percorso: 211

Numero di percorsi visitati: 31
Tempo di esecuzione: 1.6824 secondi
Memoria utilizzata: 1352 byte
Profondità massima raggiunta: 26


# BRANCH & BOUND

In [6]:
def reconstruct_path(parent, start, goal):
    """Ricostruisce il percorso a partire dal dizionario dei predecessori."""
    path = []
    node = goal
    while node is not None:
        path.append(node)
        node = parent.get(node)
    return path[::-1]  # Inverti per ottenere il percorso dal nodo di partenza a goal

def branch_and_bound(graph, start, goal, max_nodes=50000):
    """Implementazione ottimizzata di Branch and Bound per minimizzare il numero di incidenti."""
    priority_queue = [(0, start)]  # (costo incidenti, nodo attuale)
    parent = {start: None}  # Per ricostruire il percorso alla fine
    best_cost = float('inf')
    explored_paths = []  # lista di percorsi esplorati
    start_time = time.time()
    memory_before = sys.getsizeof(priority_queue) + sys.getsizeof(parent)
    max_depth = 0
    visited = set()

    while priority_queue:
        if len(explored_paths) > max_nodes:  # Evitiamo di esplorare troppi nodi
            print("Numero massimo di nodi esplorati raggiunto, interruzione anticipata!")
            break

        cost, node = heapq.heappop(priority_queue)  # Estrai il nodo con il minor costo
        path = reconstruct_path(parent, start, node)
        explored_paths.append(path)  # Salva il percorso esplorato
        visited.add(node)

        if node == goal:
            execution_time = time.time() - start_time
            memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(parent)
            memory_used = memory_after - memory_before
            return reconstruct_path(parent, start, goal), cost, explored_paths, execution_time, memory_used, max_depth

        for neighbor in graph.neighbors(node):
            if neighbor in visited:
                continue  # Evitiamo riesplorazioni inutili
            
            edge_data = graph.get_edge_data(node, neighbor)
            if isinstance(edge_data, dict):
                edge_data = min(edge_data.values(), key=lambda d: int(d.get("incidenti", float('inf'))))

            edge_cost = int(edge_data.get("incidenti", 0))
            new_cost = cost + edge_cost

            if new_cost < best_cost:  # Bounding: Scartiamo percorsi peggiori
                heapq.heappush(priority_queue, (new_cost, neighbor))
                parent[neighbor] = node  # Salviamo il predecessore
                max_depth = max(max_depth, len(parent))  # Aggiorniamo la profondità massima

    execution_time = time.time() - start_time
    memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(parent)
    memory_used = memory_after - memory_before

    return None, float('inf'), explored_paths, execution_time, memory_used, max_depth  # Ora explored_paths è una lista

# Caricare il grafo con il numero di incidenti sugli archi
G = ox.load_graphml("rete_bari_incidenti.graphml")

partenza = 270659688
arrivo = 1481415203

# Eseguire la ricerca Branch and Bound ottimizzata
shortest_path, min_incidents, explored_paths, exec_time, memory_used, max_depth = branch_and_bound(G, partenza, arrivo)

if shortest_path:
    print(f"Percorso con il minor numero di incidenti trovato: {shortest_path}")
    print(f"Numero totale di incidenti lungo il percorso: {min_incidents}")
    print(f"Numero di percorsi visitati: {len(explored_paths)}")
    print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
    print(f"Memoria utilizzata: {memory_used} byte")
    print(f"Profondità massima raggiunta: {max_depth}")
else:
    print("Nessun percorso trovato.")
    print(f"Numero di percorsi visitati: {len(explored_paths)}")
    print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
    print(f"Memoria utilizzata: {memory_used} byte")
    print(f"Profondità massima raggiunta: {max_depth}")

# RISULTATI OTTENUTI
# Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 12083787147, 569212252, 3860917204, 3841272949, 3860940020, 330076413, 279650482, 1483634369, 1483634284, 279655605, 280884748, 279380962, 339607009, 279655525, 10680697917, 316572227, 10680697906, 279655521, 9602691583, 3323511561, 1014703348, 1249973250, 4395827572, 5395447618, 5395447592, 5395448236, 5395447591, 5395447594, 5395447616, 5395448221, 5424214659, 9825165740, 9825165739, 1459967003, 5395476522, 1300778994, 1459966913, 5395475417, 5395476533, 1300778977, 1300778989, 5624550512, 330934207, 270655078, 329988596, 329988597, 270655201, 329994560, 329994583, 270654641, 1481415203]
# Numero totale di incidenti lungo il percorso: 123
# Numero di percorsi visitati: 4266
# Tempo di esecuzione: 0.0981 secondi
# Memoria utilizzata: 150992 byte
# Profondità massima raggiunta: 3561

Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 12083787147, 569212252, 3860917204, 3841272949, 3860940020, 330076413, 279650482, 1483634369, 1483634284, 279655605, 280884748, 279380962, 339607009, 279655525, 10680697917, 316572227, 10680697906, 279655521, 9602691583, 3323511561, 1014703348, 1249973250, 4395827572, 5395447618, 5395447592, 5395448236, 5395447591, 5395447594, 5395447616, 5395448221, 5424214659, 9825165740, 9825165739, 1459967003, 5395476522, 1300778994, 1459966913, 5395475417, 5395476533, 1300778977, 1300778989, 5624550512, 330934207, 270655078, 329988596, 329988597, 270655201, 329994560, 329994583, 270654641, 1481415203]
Numero totale di incidenti lungo il percorso: 123
Numero di percorsi visitati: 4266
Tempo di esecuzione: 0.0981 secondi
Memoria utilizzata: 150992 byte
Profondità massima raggiunta: 3561


# MAPPATURA DEI PERCORSI 

In [ ]:
def visualizza_percorsi(G, start, goal, explored_paths, shortest_path, output_file="percorso_b&b.html"):
    # Ottieni il centro della mappa
    center_lat, center_lon = ox.graph_to_gdfs(G, nodes=True, edges=False).geometry.y.mean(), ox.graph_to_gdfs(G, nodes=True, edges=False).geometry.x.mean()
    mappa = folium.Map(location=[center_lat, center_lon], zoom_start=13)
    
    # Aggiungi tutti i percorsi esplorati in blu
    for path in explored_paths:
        if len(path) > 1:
            coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in path]
            folium.PolyLine(coords, color="blue", weight=2, opacity=0.5).add_to(mappa)
    
    # Aggiungi il percorso ottimale in rosso (sopra il blu)
    if shortest_path:
        coords_shortest = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in shortest_path]
        folium.PolyLine(coords_shortest, color="red", weight=6, opacity=1).add_to(mappa)
    
    # Aggiungi marker per il punto di partenza e di arrivo
    folium.Marker(
        location=[G.nodes[start]['y'], G.nodes[start]['x']],
        popup="Partenza",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(mappa)
    
    folium.Marker(
        location=[G.nodes[goal]['y'], G.nodes[goal]['x']],
        popup="Arrivo",
        icon=folium.Icon(color="red", icon="flag")
    ).add_to(mappa)
    
    # Salva la mappa in un file HTML
    mappa.save(output_file)
    print(f"Mappa salvata in {output_file}")

# Esempio di utilizzo:
visualizza_percorsi(G, partenza, arrivo, explored_paths, shortest_path)


Mappa salvata in mappa_b&b.html
